In [3]:
import pandas as pd

In [4]:
# Read a file and clean it up
def cleanup_csv(file: str) -> pd.DataFrame:
    df = pd.read_csv(file)
    df = df.rename(columns={"End Year": "Year"})

    # Keep only the most recent year if there are multiple
    df = df[df["Year"] == df["Year"].max()]
    df = df[["County", "Value"]]

    return df

# Merge with good names for the columns
def merge_csv(df: pd.DataFrame, file: str, name: str):
    df = pd.merge(df, cleanup_csv(file), how="inner", on=["County"], suffixes=(None, None))
    df = df.rename(columns={"Value": name})

    return df

In [5]:
# Slightly different work for the Excel sheets
def cleanup_xlsx(file: str) -> pd.DataFrame:
    df = pd.read_excel(file)

    # Drop California aggregate row
    df = df.drop([1], axis=0)

    df = df[["Counties", "Age-adjusted rate per 100,000"]]
    df = df.rename(columns={"Counties": "County"})

    return df

def merge_xlsx(df: pd.DataFrame, file: str, name: str):
    df = pd.merge(df, cleanup_xlsx(file), how="left", on=["County"], suffixes=(None, None))
    df = df.rename(columns={"Age-adjusted rate per 100,000": name})

    return df

In [6]:
# We'll join all the county stats together

# Start with the Excel sheets
county_stats = cleanup_xlsx("../Emergency Department_Visits_Age-adjusted_rate_per_100000_2023_Counties.xlsx")
county_stats = county_stats.rename(columns={"Age-adjusted rate per 100,000": "Emergency Visits / 100000"})

county_stats = merge_xlsx(county_stats, "../Hospitalizations_Age-adjusted_rate_per_100000_2023_Counties.xlsx",
                                        "Hospitalizations / 100000")


In [7]:
# Join the CSVs
county_stats = merge_csv(county_stats, "../Avg_annual_energy_burden_percent_of_income_2018.csv",    "Energy Burden % of Income")
county_stats = merge_csv(county_stats, "../Avg_percent_of_imperviousness_2021.csv",                 "Imperviousness")
county_stats = merge_csv(county_stats, "../Distance_to_parks_half-mile_2010_2015_2020.csv",         "Park within 1/2 Mile")
county_stats = merge_csv(county_stats, "../Hospital_beds_per_10000_population_2020.csv",            "Hospital Beds / 10000")
county_stats = merge_csv(county_stats, "../Housing_built_before_1980.csv",                          "Housing Built before 1980")
county_stats = merge_csv(county_stats, "../Housing_insecurity_2022.csv",                            "Housing Insecurity")
county_stats = merge_csv(county_stats, "../Lack_of_reliable_transportation_2022.csv",               "Lack of Reliable Transportation")
county_stats = merge_csv(county_stats, "../Percent_without_internet_2018-2022.csv",                 "% w/o Internet")
county_stats = merge_csv(county_stats, "../Utility_services_threat_2022.csv",                       "Utility Services Threat")

In [9]:
# Save it
county_stats.to_excel("../County_Statistics.xlsx", index=False)

In [ ]:
# Now we will filter the Master CVI dataset
CVI_df = pd.read_excel("../Master CVI Dataset - Oct 2023.xlsx")
CVI_df = CVI_df[CVI_df.State == "CA"]

# And attach the Low Food Access data, since it's also organized by census tract
low_food_access_df = pd.read_csv("../Low_income_Low_Food_Access_by_Census_Tracts_2019_2015.csv")
low_food_access_df = low_food_access_df[low_food_access_df.Year == 2019]
low_food_access_df = low_food_access_df[["CensusTract", "Food Access"]]

CVI_df = pd.merge(CVI_df, low_food_access_df, how="left", left_on=["FIPS Code"], right_on=["CensusTract"])
CVI_df = CVI_df.drop(["State", "CensusTract"], axis=1)
1
CVI_df.to_excel("../California_CVI_dataset.xlsx", index=False)